<a href="https://colab.research.google.com/github/prananto/baud.prananto/blob/main/LSTM_Neural_Network_for_Handover_Worksheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Ini adalah modul kerja untuk desain neural network (+LSTM). Di dalam setiap step, parameter bisa dimodifikasi sebelum dieksekusi.

Silakan memodifikasi beberapa parameter neural network dan amati hasilnya.


# Step 1: Inisialisasi Library dan Download Data Set

Di langkah ini hanya dilakukan download data set dari Google Drive.
Pastikan file terunduh, terekstrak, dan tersedia di panel **Files** sebelah kiri:
1. **TDtoC2** folder training data yang berisi RSRP/RSRQ dan keberhasilan download jika UE melakukan handover ke cell 2.
2. **TDtoC3** folder training data yang berisi RSRP/RSRQ dan keberhasilan download jika UE melakukan handover ke cell 3.
3. **Test** 1000 data simulasi yang berisi RSRP/RSRQ, keberhasilan download, dan target cell jika UE melakukan handover menggunakan algoritma handover tradisional.





In [1]:
# first neural network with keras make predictions
from numpy import mean
from numpy import array_split
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout, LSTM
from keras.utils import to_categorical
from matplotlib import pyplot
from zipfile import ZipFile
#from keras.utils.vis_utils import plot_model
# download the dataset from Google Drive
#!gdown --id 1X8IuknqvH-CZOrtQTfB7AtoS7M87d1ww
#!gdown --id 1Wu4KzhGNAnKp-Xk9cXr8TQGmHckZI6Ix
#!gdown --id 1-ThmmmrDbpA7Z7O9g2B6TbaxJFbaEafj
!gdown --id 1mldXtXAkLBntRenRoWM0XOvcIV2O3v6q #3 Cells 100TD
!gdown --id 1LUJUURHtu3Rk_UX8VKmk5NeDqcNaYrJT #3 Cells 50TD
!gdown --id 1Mh9CfdZ2NVnoBvOQjIoo5t4kFwZ17mFn #3 Cells 25TD

# extract the dataset
with ZipFile('3 Cells 100TD.zip', 'r') as zObject:
    zObject.extractall(
        path="")

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1mldXtXAkLBntRenRoWM0XOvcIV2O3v6q
To: /content/3 Cells 100TD.zip
100% 490k/490k [00:00<00:00, 106MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1LUJUURHtu3Rk_UX8VKmk5NeDqcNaYrJT
To: /content/3 Cells 50TD.zip
100% 446k/446k [00:00<00:00, 26.0MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.

In [ ]:
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

oneRow = load_file('Test Data just one row.txt')
print (oneRow.shape)
splitted = array_split(oneRow,6, axis=1)
RSRP_Cell1 = splitted[0]
RSRP_Cell2 = splitted[1]
RSRP_Cell3 = splitted[2]
RSRQ_Cell1 = splitted[3]
RSRQ_Cell2 = splitted[4]
RSRQ_Cell3 = splitted[5]
#load one-row test
oneRowTestX = dstack(splitted)





(1, 180)


#Step 2: Loading Data Set

Pastikan variabel trainXtoC2, trainYtoC2, trainXtoC3, trainYtoC3, testX, testY, targetCellTrad tersedia di panel **Variables**.

Jumlah sampel RSRP/RSRQ yang diambil dapat diatur pada fungsi load_file (30, 10, atau 3 sampel RSRP/RSRQ terakhir).

In [7]:
# load a single file as a numpy array
def load_file(filepath):

	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	if 'input' in filepath:
		dataframe = dataframe.truncate(axis=1, before=0, after=29) # 30 RSRP/RSRQ samples
		#dataframe = dataframe.truncate(axis=1, before=20, after=29) # 10 RSRP/RSRQ sample
		#dataframe = dataframe.truncate(axis=1, before=27, after=29) # 3 RSRP/RSRQ sample
	return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/input/'
	# load all 6 files as a single array
	filenames = list()
	# RSRP
	filenames += ['RSRP_Cell1_'+group+'.txt', 'RSRP_Cell2_'+group+'.txt', 'RSRP_Cell3_'+group+'.txt']
	# RSRQ
	filenames += ['RSRQ_Cell1_'+group+'.txt', 'RSRQ_Cell2_'+group+'.txt', 'RSRQ_Cell3_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	Y = load_file(prefix + group + '/Y_'+group+'.txt')
	return X, Y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainXtoC2, trainYtoC2 = load_dataset_group('TDtoC2', prefix + '')
	#print(trainXtoC2.shape, trainYtoC2.shape)
	trainXtoC3, trainYtoC3 = load_dataset_group('TDtoC3', prefix + '')
	#print(trainXtoC3.shape, trainYtoC3.shape)
	# load all test
	testX, testY = load_dataset_group('Test', prefix + '')
	#print(testX.shape, testY.shape)
	targetCellTrad = load_file(prefix + 'Test/Y_Trad.txt')
	#print(trainXtoC2.shape, trainYtoC2.shape, testX.shape, testY.shape)
	#print(trainXtoC3.shape, trainYtoC3.shape, testX.shape, testY.shape)
	return trainXtoC2, trainYtoC2, trainXtoC3, trainYtoC3, testX, testY, targetCellTrad

trainXtoC2, trainYtoC2, trainXtoC3, trainYtoC3, testX, testY, targetCellTrad = load_dataset()

# Step 3: Pembuatan Model Neural Network

Dalam langkah ini, dibuat 2 model NN:
1. **modelC2**: Model LSTM-NN untuk menentukan apakah download berhasil jika target cell adalah cell 2.
2. **modelC3**: Model LSTM-NN untuk menentukan apakah download berhasil jika target cell adalah cell 3.

Struktur kedua model ini adalah sama:
1. **LSTM Layer**: Rangkaian LSTM Cell yang mengolah data secara serial, 18 timestep terakhir akan dimasukkan ke layer MLP (neural network biasa).
2. **Dropout**: teknik regularisasi yang membuang sebagian data yang diolah pada RNN, dalam hal ini 50%.
3. **18 input node**: 3 sampel RSRP dan RSRQ dari semua cell yang ada dalam jaringan (cell 1, cell 2, dan cell 3).
2. **4 hidden node**.
3. **1 output node**: download berhasil (1) atau tidak (0) jika dilakukan handover ke cell tersebut.



In [8]:
verbose, epochs, batch_size = 1, 150, 10
n_timesteps, n_features, n_outputs = trainXtoC2.shape[1], trainXtoC2.shape[2], trainYtoC2.shape[1]
modelC2 = Sequential()
modelC2.add(LSTM(18, input_shape=(n_timesteps,n_features)))
modelC2.add(Dropout(0.5))
modelC2.add(Dense(4, input_shape=(18,), activation='relu'))
modelC2.add(Dense(n_outputs, activation='sigmoid'))
modelC2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#plot_model(modelC2, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

modelC3 = Sequential()
modelC3.add(LSTM(18, input_shape=(n_timesteps,n_features)))
modelC3.add(Dropout(0.5))
modelC3.add(Dense(4, input_shape=(18,), activation='relu'))
modelC3.add(Dense(n_outputs, activation='sigmoid'))
modelC3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 4: Proses Training

Dalam langkah ini dilakukan proses iterasi training data. Dalam langkah ini dapat dicoba melakukan variasi pada hyperparamater training:
1. **Epoch**: One pass through all of the rows in the training dataset
2. **Batch**: One or more samples considered by the model within an epoch before weights are updated


In [9]:
# fit network
modelC2.fit(trainXtoC2, trainYtoC2, epochs=epochs, batch_size=batch_size, verbose=verbose)
modelC3.fit(trainXtoC3, trainYtoC3, epochs=epochs, batch_size=batch_size, verbose=verbose)
# evaluate model
#_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
#return accuracy

Epoch 1/150
10/10 [==============================] - 3s 24ms/step - loss: 0.6957 - accuracy: 0.4500
Epoch 2/150
10/10 [==============================] - 0s 10ms/step - loss: 0.6840 - accuracy: 0.5400
Epoch 3/150
10/10 [==============================] - 0s 10ms/step - loss: 0.6835 - accuracy: 0.4000
Epoch 4/150
10/10 [==============================] - 0s 12ms/step - loss: 0.7105 - accuracy: 0.4100
Epoch 5/150
10/10 [==============================] - 0s 11ms/step - loss: 0.6911 - accuracy: 0.4700
Epoch 6/150
10/10 [==============================] - 0s 14ms/step - loss: 0.6967 - accuracy: 0.5100
Epoch 7/150
10/10 [==============================] - 0s 11ms/step - loss: 0.6852 - accuracy: 0.5700
Epoch 8/150
10/10 [==============================] - 0s 10ms/step - loss: 0.6597 - accuracy: 0.6200
Epoch 9/150
10/10 [==============================] - 0s 11ms/step - loss: 0.7129 - accuracy: 0.4200
Epoch 10/150
10/10 [==============================] - 0s 11ms/step - loss: 0.7037 - accuracy: 0.5100

# Step 5: Evaluasi Akurasi Model

Dalam langkah ini dapat diukur akurasi model yang sudah dilatih. Akurasi diukur dengan memasukkan input training data yang sama ke dalam model, kemudian outputnya dibandingkan dengan output training data.   

Dapat diperhatikan bahwa untuk setiap proses iterasi training data (Step 3), akurasi model bisa berbeda-beda.

In [10]:
# evaluate the keras model
_, accuracy = modelC2.evaluate(trainXtoC2, trainYtoC2)
print('Accuracy of modelC2: %.2f' % (accuracy*100))
_, accuracy = modelC3.evaluate(trainXtoC3, trainYtoC3)
print('Accuracy of modelC3: %.2f' % (accuracy*100))

4/4 [==============================] - 1s 306ms/step - loss: 0.1852 - accuracy: 0.9900
Accuracy of modelC2: 99.00
4/4 [==============================] - 2s 124ms/step - loss: 0.5245 - accuracy: 0.7400
Accuracy of modelC3: 74.00


# Step 6: Melakukan Prediksi

Dalam langkah ini akan ditentukan kemungkinan keberhasilan download file jika handover dilakukan ke **cell 2** (menggunakan **modelC2**) atau ke **cell 3** (menggunakan **modelC3**). Hasilnya adalah angka floating point antara 0 dan 1.

In [11]:
# make class predictions with the model
predictionsC2 = modelC2.predict(testX)
predictionsC3 = modelC3.predict(testX)
# summarize
for i in range(10):
	print("Prediction if HO to Cell 2, simulation case %d: Result %f, rounded to %d  " % (i, predictionsC2[i], (predictionsC2[i] > 0.5).astype(int)))
	print("Prediction if HO to Cell 3, simulation case %d: Result %f, rounded to %d  " % (i, predictionsC3[i], (predictionsC3[i] > 0.5).astype(int)))

32/32 [==============================] - 2s 32ms/step
Prediction if HO to Cell 2, simulation case 0: Result 0.927168, rounded to 1  
Prediction if HO to Cell 3, simulation case 0: Result 0.660653, rounded to 1  
Prediction if HO to Cell 2, simulation case 1: Result 0.698934, rounded to 1  
Prediction if HO to Cell 3, simulation case 1: Result 0.614578, rounded to 1  
Prediction if HO to Cell 2, simulation case 2: Result 0.624631, rounded to 1  
Prediction if HO to Cell 3, simulation case 2: Result 0.614577, rounded to 1  
Prediction if HO to Cell 2, simulation case 3: Result 0.948346, rounded to 1  
Prediction if HO to Cell 3, simulation case 3: Result 0.581056, rounded to 1  
Prediction if HO to Cell 2, simulation case 4: Result 0.487632, rounded to 0  
Prediction if HO to Cell 3, simulation case 4: Result 0.614600, rounded to 1  
Prediction if HO to Cell 2, simulation case 5: Result 0.948410, rounded to 1  
Prediction if HO to Cell 3, simulation case 5: Result 0.581053, rounded to 1 

# Step 7: Menentukan Target Cell

Dari rekomendasi kedua model di Step 5, dapat ditentukan target cell. Target cell (cell 2 atau cell 3) dapat ditentukan dengan membandingkan hasil prediksi kedua model NN.

In [12]:
targetCellNN = targetCellTrad.copy()
i=0
while i < targetCellTrad.size:
  if predictionsC2[i] > predictionsC3[i]:
    targetCellNN[i] = 2
  else:
    targetCellNN[i] = 3
  print("Target cell for simulation case %d is Cell %d" % (i, targetCellNN[i]))
  i +=1

Target cell for simulation case 0 is Cell 2
Target cell for simulation case 1 is Cell 2
Target cell for simulation case 2 is Cell 2
Target cell for simulation case 3 is Cell 2
Target cell for simulation case 4 is Cell 3
Target cell for simulation case 5 is Cell 2
Target cell for simulation case 6 is Cell 2
Target cell for simulation case 7 is Cell 3
Target cell for simulation case 8 is Cell 2
Target cell for simulation case 9 is Cell 3
Target cell for simulation case 10 is Cell 3
Target cell for simulation case 11 is Cell 3
Target cell for simulation case 12 is Cell 3
Target cell for simulation case 13 is Cell 3
Target cell for simulation case 14 is Cell 3
Target cell for simulation case 15 is Cell 3
Target cell for simulation case 16 is Cell 3
Target cell for simulation case 17 is Cell 2
Target cell for simulation case 18 is Cell 3
Target cell for simulation case 19 is Cell 3
Target cell for simulation case 20 is Cell 2
Target cell for simulation case 21 is Cell 3
Target cell for simu

# Step 8: Menentukan Tingkat Keberhasilan Download

Karena jaringan sangat sederhana (hanya 3 cell dan 2 kemungkinan target cell), maka tingkat keberhasilan download jika target cell ditentukan oleh NN bisa dikalkulasi dengan aturan sederhana ini:

| Berhasil Download dengan Algoritma Tradisional | Target Cell antara Algoritma Tradisional dan NN Sama | Berhasil Download dengan NN |
| --- | --- | --- |
| TIDAK | TIDAK | YA |
| TIDAK | YA | TIDAK |
| YA | TIDAK | TIDAK |
| YA | YA | YA |


In [13]:
yNN = testY.copy()
i=0
while i < targetCellTrad.size:
  if testY[i]==0: #Download failed using traditional algorithm
    if targetCellNN[i]==targetCellTrad[i]: #Target cells are SAME
      yNN[i]=0
    else: #Target cells are DIFFERENT
      yNN[i]=1
  else: #Download successful using traditional algorithm
    if targetCellNN[i]==targetCellTrad[i]: #Target cells are SAME
      yNN[i]=1
    else:
      yNN[i]=0
  print("For simulation case %d download success %d" % (i, yNN[i]))
  i +=1


For simulation case 0 download success 1
For simulation case 1 download success 1
For simulation case 2 download success 1
For simulation case 3 download success 1
For simulation case 4 download success 1
For simulation case 5 download success 1
For simulation case 6 download success 1
For simulation case 7 download success 1
For simulation case 8 download success 1
For simulation case 9 download success 1
For simulation case 10 download success 1
For simulation case 11 download success 1
For simulation case 12 download success 1
For simulation case 13 download success 1
For simulation case 14 download success 1
For simulation case 15 download success 1
For simulation case 16 download success 1
For simulation case 17 download success 1
For simulation case 18 download success 0
For simulation case 19 download success 1
For simulation case 20 download success 1
For simulation case 21 download success 0
For simulation case 22 download success 1
For simulation case 23 download success 1
Fo

# Step 8: Perbandingan Tingkat Keberhasilan Download

Dalam langkah ini akan dihutung tingkat keberhasilan download file jika menggunakan algoritma handover tradisional dan jika menggunakan neural network.

In [14]:
print("Download success rate for Traditional Algorithm: %f" % (testY.sum()/testY.size))
print("Download success rate for Neural Network: %f" % (yNN.sum()/yNN.size))

Download success rate for Traditional Algorithm: 0.862000
Download success rate for Neural Network: 0.900000
